In [1]:
import pandas as pd
import numpy as np

In [2]:
column_names = ['Ticker','Date_YMD','Time','open','high','low','close','volume','extra']

# here add your csv file path
df = pd.read_csv('../nifty-banknifty-data/Nifty-index-data/2021/Yearly Consolidated/NIFTY.csv',names=column_names,header=None)

df

,Ticker,Date_YMD,Time,open,high,low,close,volume,extra
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2,0,0
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0,0,0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7,0,0
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2,0,0
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5,0,0
...,...,...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3,0,0
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3,0,0
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8,0,0
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6,0,0


In [3]:
df.drop(columns=['volume','extra'],inplace=True)
df



,Ticker,Date_YMD,Time,open,high,low,close
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5
...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6


In [4]:
# x = df.where((df['Time'] == '09:16') | (df['Time'] == '15:30'))
# x = x.dropna()
x = df.copy()
x['DateTime'] = pd.to_datetime(x['Date_YMD'] + ' ' + x['Time'])
x.set_index('DateTime', inplace=True)
resampled_df = x.resample('15min').agg({
    'Ticker' : 'first',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})
# print(resampled_df.shape)
resampled_df.dropna(inplace=True)
temp = resampled_df.loc['2021-08-02 09:15:00':'2021-08-02 15:30:00',:]
print(len(temp))

resampled_df

26


,Ticker,open,high,low,close
DateTime,,,,,
2021-01-01 09:15:00,NIFTY,13997.9,14023.5,13991.3,14018.5
2021-01-01 09:30:00,NIFTY,14018.0,14028.9,14001.2,14024.0
2021-01-01 09:45:00,NIFTY,14024.2,14033.8,14022.5,14032.2
2021-01-01 10:00:00,NIFTY,14032.3,14033.3,14016.3,14025.8
2021-01-01 10:15:00,NIFTY,14025.0,14036.5,14017.6,14035.5
...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17353.5,17369.3,17351.4,17368.8
2021-12-31 14:45:00,NIFTY,17369.3,17377.3,17358.1,17362.8
2021-12-31 15:00:00,NIFTY,17363.5,17365.2,17341.8,17356.6


In [5]:
new_df = resampled_df.copy()

#Heiken Ashi Close
new_df['HA_Close'] = (new_df['open'] + new_df['close'] + new_df['high'] + new_df['low'])/4


#Heiken Ashi Open
new_df['HA_Open'] = pd.Series([float('nan') * len(new_df)])
new_df.iloc[0,new_df.columns.get_loc('HA_Open')] = (new_df.iloc[0,new_df.columns.get_loc('open')] + new_df.iloc[0,new_df.columns.get_loc('close')]) / 2

for i in range(1, len(new_df)):
    new_df.iloc[i, new_df.columns.get_loc('HA_Open')] = (new_df.iloc[i - 1]['HA_Open'] + new_df.iloc[i - 1]['HA_Close']) / 2


#Heiken Ashi High
new_df['HA_High'] = new_df[['high', 'HA_Open', 'HA_Close']].max(axis=1)

#Heiken Ashi Low
new_df['HA_Low'] = new_df[['low', 'HA_Open', 'HA_Close']].min(axis=1)

new_df


,Ticker,open,high,low,close,HA_Close,HA_Open,HA_High,HA_Low
DateTime,,,,,,,,,
2021-01-01 09:15:00,NIFTY,13997.9,14023.5,13991.3,14018.5,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,NIFTY,14018.0,14028.9,14001.2,14024.0,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,NIFTY,14024.2,14033.8,14022.5,14032.2,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,NIFTY,14032.3,14033.3,14016.3,14025.8,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,NIFTY,14025.0,14036.5,14017.6,14035.5,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17353.5,17369.3,17351.4,17368.8,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,NIFTY,17369.3,17377.3,17358.1,17362.8,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,NIFTY,17363.5,17365.2,17341.8,17356.6,17356.775,17364.495387,17365.2,17341.8000


In [6]:
new_df.drop(['open','high','low','close'],axis=1,inplace=True)
new_df.columns = ['Ticker','close','open','high','low']
new_df



,Ticker,close,open,high,low
DateTime,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,NIFTY,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,NIFTY,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,NIFTY,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,NIFTY,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,NIFTY,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,NIFTY,17356.775,17364.495387,17365.2,17341.8000


In [7]:
new_df['type'] = new_df.apply(lambda row: 'call' if row['close'] > row['open'] else 'put', axis=1)

new_df

,Ticker,close,open,high,low,type
DateTime,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.5,13991.3000,put
2021-01-01 09:30:00,NIFTY,14018.025,14008.000000,14028.9,14001.2000,call
2021-01-01 09:45:00,NIFTY,14028.175,14013.012500,14033.8,14013.0125,call
2021-01-01 10:00:00,NIFTY,14026.925,14020.593750,14033.3,14016.3000,call
2021-01-01 10:15:00,NIFTY,14028.650,14023.759375,14036.5,14017.6000,call
...,...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17360.750,17363.481548,17369.3,17351.4000,put
2021-12-31 14:45:00,NIFTY,17366.875,17362.115774,17377.3,17358.1000,call
2021-12-31 15:00:00,NIFTY,17356.775,17364.495387,17365.2,17341.8000,put


In [8]:

def check_time(index):
    return (index.time() == pd.Timestamp('09:15:00').time()) | (index.time() == pd.Timestamp('15:15:00').time())

mask = new_df.index.map(check_time)

flc = new_df[mask]
flc

,Ticker,close,open,high,low,type
DateTime,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put
...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call


In [9]:
def calculate_result(row):
    previous_row = flc.shift(1).loc[row.name]  
    if pd.isnull(previous_row).any():  
        return np.NAN
    else:
        return row['type'] ==  previous_row['type']

flc.loc[:,'result'] = flc.apply(calculate_result, axis=1)
flc

/tmp/ipykernel_31632/562910030.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc.loc[:,'result'] = flc.apply(calculate_result, axis=1)


,Ticker,close,open,high,low,type,result
DateTime,,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put,NaN
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put,True
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call,True
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False
...,...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put,True
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put,True
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call,False


In [10]:
def calculate_gain(row):
    previous_row = flc.shift(1).loc[row.name]  
    if pd.isnull(previous_row).any():  
        print(row.name)
        return 0
    else:
        flc.loc[row.name,'buy_price'] = previous_row['close']
        return row['open'] - previous_row['close']

flc.loc[:,'gain'] = flc.apply(calculate_gain, axis=1)
flc.loc[:,'gain'] = flc['gain'].abs()
flc

2021-01-01 09:15:00
2021-01-01 15:15:00


/tmp/ipykernel_31632/3182690955.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc.loc[:,'gain'] = flc.apply(calculate_gain, axis=1)


,Ticker,close,open,high,low,type,result,buy_price,gain
DateTime,,,,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.500000,13991.300000,put,NaN,NaN,0.000000
2021-01-01 15:15:00,NIFTY,14015.425,14035.185230,14035.185230,14001.600000,put,True,NaN,0.000000
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,14015.425,4.540058
2021-01-04 15:15:00,NIFTY,14134.975,14115.239597,14148.000000,14115.239597,call,True,14099.375,15.864597
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,14134.975,0.683851
...,...,...,...,...,...,...,...,...,...
2021-12-29 15:15:00,NIFTY,17201.250,17239.863031,17239.863031,17176.700000,put,True,17221.800,18.063031
2021-12-30 09:15:00,NIFTY,17191.950,17203.290758,17217.100000,17146.300000,put,True,17201.250,2.040758
2021-12-30 15:15:00,NIFTY,17215.425,17212.681670,17226.300000,17197.800000,call,False,17191.950,20.731670


In [11]:
flc_t = flc[flc['result'] == True].copy()
flc_f = flc[flc['result'] == False].copy()

flc_f

,Ticker,close,open,high,low,type,result,buy_price,gain
DateTime,,,,,,,,,
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,14015.425,4.540058
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,14134.975,0.683851
2021-01-05 15:15:00,NIFTY,14204.500,14177.093966,14215.600000,14177.093966,call,False,14086.925,90.168966
2021-01-07 15:15:00,NIFTY,14134.525,14169.849694,14169.849694,14123.100000,put,False,14238.550,68.700306
2021-01-08 09:15:00,NIFTY,14246.425,14147.743674,14259.200000,14147.743674,call,False,14134.525,13.218674
...,...,...,...,...,...,...,...,...,...
2021-12-24 15:15:00,NIFTY,17004.400,17020.186838,17020.186838,16996.400000,put,False,17113.600,93.413162
2021-12-27 15:15:00,NIFTY,17087.825,17085.927284,17095.300000,17079.300000,call,False,16912.675,173.252284
2021-12-29 09:15:00,NIFTY,17221.800,17228.501667,17255.800000,17180.900000,put,False,17234.675,6.173333


In [12]:
mask = flc_t.index.map(lambda x : x.time() == pd.Timestamp('09:15:00').time())
mask2 = flc_f.index.map(lambda x : x.time() == pd.Timestamp('09:15:00').time())
flc_t = flc_t[mask]
flc_f = flc_f[mask2]
flc_t


,Ticker,close,open,high,low,type,result,buy_price,gain
DateTime,,,,,,,,,
2021-01-06 09:15:00,NIFTY,14226.975,14194.498491,14244.200000,14194.498491,call,True,14204.500,10.001509
2021-01-07 09:15:00,NIFTY,14238.550,14134.056702,14256.300000,14134.056702,call,True,14138.725,4.668298
2021-01-11 09:15:00,NIFTY,14439.825,14341.268994,14458.700000,14341.268994,call,True,14353.800,12.531006
2021-01-18 09:15:00,NIFTY,14426.325,14434.590471,14459.200000,14389.200000,put,True,14433.175,1.415471
2021-01-22 09:15:00,NIFTY,14575.925,14623.403746,14623.403746,14545.100000,put,True,14602.675,20.728746
...,...,...,...,...,...,...,...,...,...
2021-12-24 09:15:00,NIFTY,17113.600,17060.955702,17155.600000,17051.400000,call,True,17068.650,7.694298
2021-12-27 09:15:00,NIFTY,16912.675,17005.284209,17005.284209,16833.200000,put,True,17004.400,0.884209
2021-12-28 09:15:00,NIFTY,17184.450,17091.925571,17203.900000,17091.925571,call,True,17087.825,4.100571


In [13]:
total_gain = flc_t['gain'].sum()
total_lose = flc_f['gain'].sum()

print(f"Gain : {total_gain}")
print(f"Lose : {total_lose}")

Gain : 815.8584514338727
Lose : 657.4321175554505


In [14]:
flc_f

,Ticker,close,open,high,low,type,result,buy_price,gain
DateTime,,,,,,,,,
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,14015.425,4.540058
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,14134.975,0.683851
2021-01-08 09:15:00,NIFTY,14246.425,14147.743674,14259.200000,14147.743674,call,False,14134.525,13.218674
2021-01-12 09:15:00,NIFTY,14472.500,14482.996013,14487.700000,14432.800000,put,False,14487.725,4.728987
2021-01-13 09:15:00,NIFTY,14634.025,14569.780572,14645.800000,14569.780572,call,False,14565.675,4.105572
...,...,...,...,...,...,...,...,...,...
2021-12-10 09:15:00,NIFTY,17495.075,17503.187167,17534.300000,17434.800000,put,False,17512.050,8.862833
2021-12-15 09:15:00,NIFTY,17307.550,17312.870647,17351.200000,17262.200000,put,False,17319.800,6.929353
2021-12-16 09:15:00,NIFTY,17347.275,17221.817211,17379.300000,17221.817211,call,False,17213.175,8.642211


In [15]:
flc.to_csv('output.csv')

In [16]:
new_df

,Ticker,close,open,high,low,type
DateTime,,,,,,
2021-01-01 09:15:00,NIFTY,14007.800,14008.200000,14023.5,13991.3000,put
2021-01-01 09:30:00,NIFTY,14018.025,14008.000000,14028.9,14001.2000,call
2021-01-01 09:45:00,NIFTY,14028.175,14013.012500,14033.8,14013.0125,call
2021-01-01 10:00:00,NIFTY,14026.925,14020.593750,14033.3,14016.3000,call
2021-01-01 10:15:00,NIFTY,14028.650,14023.759375,14036.5,14017.6000,call
...,...,...,...,...,...,...
2021-12-31 14:30:00,NIFTY,17360.750,17363.481548,17369.3,17351.4000,put
2021-12-31 14:45:00,NIFTY,17366.875,17362.115774,17377.3,17358.1000,call
2021-12-31 15:00:00,NIFTY,17356.775,17364.495387,17365.2,17341.8000,put


In [17]:
# Resample new_df to create 1-day candles
one_day_candle = new_df.resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})
one_day_candle.dropna(inplace=True)

one_day_candle['type'] = one_day_candle.apply(lambda row: 'call' if row['close'] > row['open'] else 'put', axis=1)

one_day_candle.head(40)

,open,high,low,close,type
DateTime,,,,,
2021-01-01,14008.200000,14049.800000,13991.300000,14014.625,call
2021-01-04,14019.965058,14148.000000,13953.800000,14143.475,call
2021-01-05,14134.291149,14215.600000,14048.200000,14198.200,call
2021-01-06,14194.498491,14244.200000,14039.900000,14130.875,put
2021-01-07,14134.056702,14256.300000,14123.100000,14143.300,call
2021-01-08,14147.743674,14367.300000,14147.743674,14346.275,call
2021-01-11,14341.268994,14498.200000,14341.268994,14484.925,call
2021-01-12,14482.996013,14590.700000,14432.800000,14572.550,call
2021-01-13,14569.780572,14653.300000,14435.700000,14557.100,put


In [18]:
flc.head(20)
flc_f.head(15)

,Ticker,close,open,high,low,type,result,buy_price,gain
DateTime,,,,,,,,,
2021-01-04 09:15:00,NIFTY,14099.375,14019.965058,14107.800000,14019.965058,call,False,14015.425,4.540058
2021-01-05 09:15:00,NIFTY,14086.925,14134.291149,14134.291149,14048.200000,put,False,14134.975,0.683851
2021-01-08 09:15:00,NIFTY,14246.425,14147.743674,14259.200000,14147.743674,call,False,14134.525,13.218674
2021-01-12 09:15:00,NIFTY,14472.500,14482.996013,14487.700000,14432.800000,put,False,14487.725,4.728987
2021-01-13 09:15:00,NIFTY,14634.025,14569.780572,14645.800000,14569.780572,call,False,14565.675,4.105572
2021-01-14 09:15:00,NIFTY,14543.375,14555.744276,14562.800000,14524.100000,put,False,14561.050,5.305724
2021-01-15 09:15:00,NIFTY,14589.100,14594.090273,14617.500000,14569.300000,put,False,14593.200,0.890273
2021-01-19 09:15:00,NIFTY,14389.400,14261.758483,14419.500000,14261.758483,call,False,14263.450,1.691517
2021-01-20 09:15:00,NIFTY,14558.075,14522.624549,14587.100000,14517.500000,call,False,14520.475,2.149549


In [19]:
# lose recovered by waiting for 2 day

def fun(row):
    today = row.name.date()
    timestamp = pd.Timestamp(today)
    # print(timestamp)
    present_day = one_day_candle.loc[timestamp]
    next_day = one_day_candle.shift(-1).loc[timestamp] 
    if pd.isnull(next_day).any(): 
        return (0,False)
    else:
        if row['type'] == 'put': # means you have bought call on previuos day 
            if (next_day['high'] >= row['buy_price']) | (present_day['high'] >= row['buy_price']):
                # print(row.name.date())
                return (row['gain'],True)
        else: # means you have bought put on previuos day
            if (next_day['low'] <= row['buy_price']) | (present_day['low'] <= row['buy_price']):
                return (row['gain'],True)
        return (0,False)
    
recovered_lose = flc_f.apply(fun,axis=1)

total_recovered_lose = 0
for t in recovered_lose:
    total_recovered_lose = total_recovered_lose + t[0]

total_recovered_lose


395.7438542119835

In [20]:
print(total_gain)
print(total_lose)
print(total_recovered_lose)

815.8584514338727
657.4321175554505
395.7438542119835


In [21]:
total_updated_lose = total_lose - total_recovered_lose
total_updated_lose

261.68826334346704

In [22]:
#extra lose because of waiting for 2 day
extra_lose = pd.DataFrame(recovered_lose,columns=['data'])

def fun2(row):
    if row['data'][1] == False:
        buyPrice = flc_f.loc[row.name]['buy_price']
        sellPrice = one_day_candle.shift(-1).loc[pd.Timestamp(row.name.date())]['close']
        # print(f'buy price at {row.name} is {buyPrice} and sellprice is {sellPrice}')
        return abs(sellPrice-buyPrice)
    else:
        return 0

extra_lose = extra_lose.apply(fun2,axis=1)
total_extra_lose = extra_lose.sum()
total_extra_lose

7913.874999999991

In [23]:
print('For buying at 3:30 yesterday and selling at today 9:15 no matter what')
print(f'Total Gain: {total_gain}')
print(f'Total lose: {total_lose}')

print('\nTotal recovered lose because of waiting till tomorrow end ')
print(f'Total recovered_lose: {total_recovered_lose}')
print(f'Total Updated lose = total lose - total recovered lose = : {total_updated_lose}')

print('\nTotal extra lose because of waiting till tomorrow end')
print(f'Total extra lose : {total_extra_lose}')

print('NOTE: This extra lose is because of the continues reverese trend of market then our buy item for 2 days')

For buying at 3:30 yesterday and selling at today 9:15 no matter what
Total Gain: 815.8584514338727
Total lose: 657.4321175554505

Total recovered lose because of waiting till tomorrow end 
Total recovered_lose: 395.7438542119835
Total Updated lose = total lose - total recovered lose = : 261.68826334346704

Total extra lose because of waiting till tomorrow end
Total extra lose : 7913.874999999991
NOTE: This extra lose is because of the continues reverese trend of market then our buy item for 2 days
